In [1]:
# Bring in (new) PNC cohort

import pickle
from pathlib import Path
import numpy as np

newdir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/BSNIP'
newdemo = pickle.load(open(f'{newdir}/demographics.pkl', 'rb'))

newfc = dict()

for sub in newdemo['Age_cal']:
    f = f'{newdir}/fc/{sub}_task-unk_fc.npy'
    if not Path(f).exists():
        continue
    p = np.load(f)
    newfc[f'{sub}'] = p

print(len(newfc))

1244


In [2]:
# Angle estimate

import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt

pi = 3.14
pi2 = 2*pi

def tonp(t):
    return t.detach().cpu().numpy()

class AngleBasis(nn.Module):
    def __init__(self, mixn):
        super(AngleBasis, self).__init__()
        self.mixn = mixn
        self.thetas = nn.Parameter((pi*torch.rand(self.mixn,264)+pi/2).float().cuda())
        self.jitter = nn.Parameter(torch.ones(self.mixn,264).float().cuda())
        
    def project(self):
        with torch.no_grad():
            self.jitter[self.jitter < 0] = 0
            self.jitter[self.jitter > 1] = 1
        
    def phases(self):
        t0 = self.thetas.unsqueeze(2)
        t1 = self.thetas.unsqueeze(1)
        return t0-t1
    
    def jit(self):
        j0 = self.jitter.unsqueeze(2)
        j1 = self.jitter.unsqueeze(1)
        return j0*j1
    
    def ps(self, jitter=True):
        t = self.phases()
        p = torch.cos(t)
        if jitter:
            j = self.jit()
            p = j*p
        return p
    
    def dump(self):
        return dict(mixn=self.mixn, thetas=tonp(self.thetas), jitter=tonp(self.jitter))
    
    def psum(self):
        return torch.mean(self.ps(), axis=0)
    
    def pvec(self):
        a,b = np.triu_indices(264,1)
        p = self.psum()
        return p[a,b]
    
print('Complete')

Complete


In [11]:
nepochs = 500
pperiod = 500

def fromnp(x):
    return torch.from_numpy(x).float().cuda()

def rmse(yhat, y):
    if isinstance(yhat, np.ndarray):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

bases = dict()
i = 0

for subtask in newfc:
    x = fromnp(newfc[subtask])

    min_loss = 10
    sav = None

    for _ in range(5):
        basis = AngleBasis(1)
        optim = torch.optim.Adam(basis.parameters(), lr=1e-2, weight_decay=0)

        for e in range(nepochs):
            optim.zero_grad()
            xhat = basis.pvec()
            loss = rmse(xhat, x)
            loss.backward()
            optim.step()
            basis.project()
            if e == nepochs-1 or e % pperiod == 1:
                print(f'{e} {float(loss)}')

        if loss < min_loss:
            print('Saving')
            min_loss = float(loss)
            sav = basis
    
    bases[subtask] = sav.dump()
    i += 1
    
    print(f'Finished {i} {subtask}')
        
print('Complete')

1 0.6277095675468445
499 0.14070093631744385
Saving
1 0.6299174427986145
499 0.1400204747915268
Saving
1 0.6319666504859924
499 0.14102692902088165
1 0.6467759609222412
499 0.13971304893493652
Saving
1 0.6255670189857483
499 0.14036299288272858
Finished 1 Baltimore.S0015SRH1
1 0.6924705505371094
499 0.14454878866672516
Saving
1 0.6896330118179321
499 0.1464594304561615
1 0.6910004615783691
499 0.1512811779975891
1 0.6861085295677185
499 0.14272037148475647
Saving
1 0.6920998096466064
499 0.1478995680809021
Finished 2 Baltimore.S0021FHS1
1 0.5944428443908691
499 0.10124826431274414
Saving
1 0.600387692451477
499 0.10146138072013855
1 0.6017574667930603
499 0.10144421458244324
1 0.5921037197113037
499 0.10122530907392502
Saving
1 0.6066908240318298
499 0.10123267024755478
Finished 3 Baltimore.S0172LAC1
1 0.6760510802268982
499 0.16757097840309143
Saving
1 0.675036609172821
499 0.16620083153247833
Saving
1 0.6643794775009155
499 0.16466489434242249
Saving
1 0.6702779531478882
499 0.157521

499 0.10990861058235168
Saving
1 0.6409863233566284
499 0.1102588102221489
1 0.6432837247848511
499 0.11033547669649124
1 0.6348459720611572
499 0.11133705824613571
Finished 31 Baltimore.S1109FMB1
1 0.6545588374137878
499 0.11278937757015228
Saving
1 0.6277291178703308
499 0.11299236863851547
1 0.637799859046936
499 0.11206503212451935
Saving
1 0.6289693713188171
499 0.11273300647735596
1 0.6276634931564331
499 0.11206851154565811
Finished 32 Baltimore.S1109WGJ1
1 0.6465842723846436
499 0.1445871889591217
Saving
1 0.6533905267715454
499 0.14598262310028076
1 0.6513903737068176
499 0.1452878713607788
1 0.6679031848907471
499 0.14561358094215393
1 0.6544705629348755
499 0.14615759253501892
Finished 33 Baltimore.S1132EWE1
1 0.598386287689209
499 0.11232759058475494
Saving
1 0.6143080592155457
499 0.11089956015348434
Saving
1 0.6347275972366333
499 0.11089444905519485
Saving
1 0.5980212092399597
499 0.11113332957029343
1 0.6108260154724121
499 0.11096733063459396
Finished 34 Baltimore.S113

499 0.12222806364297867
1 0.6049326062202454
499 0.1211731806397438
1 0.6162216067314148
499 0.12018251419067383
1 0.5996034145355225
499 0.1214304119348526
Finished 61 Baltimore.S2514DUC1
1 0.6051090955734253
499 0.10372748970985413
Saving
1 0.6172754764556885
499 0.10375227779150009
1 0.5995362401008606
499 0.10369337350130081
Saving
1 0.6204215884208679
499 0.10349489748477936
Saving
1 0.6159119606018066
499 0.10354001820087433
Finished 62 Baltimore.S2519DTD1
1 0.6176815629005432
499 0.1177707314491272
Saving
1 0.6269308924674988
499 0.11837589740753174
1 0.6470268964767456
499 0.11818410456180573
1 0.6258087158203125
499 0.11612222343683243
Saving
1 0.6347336173057556
499 0.1173839345574379
Finished 63 Baltimore.S2520BMI1
1 0.6287739276885986
499 0.13327765464782715
Saving
1 0.6224622130393982
499 0.13353241980075836
1 0.6338162422180176
499 0.13986162841320038
1 0.6203618049621582
499 0.1318204402923584
Saving
1 0.6254389882087708
499 0.13233062624931335
Finished 64 Baltimore.S252

499 0.12301197648048401
1 0.611455500125885
499 0.12313006073236465
1 0.6062020063400269
499 0.12296270579099655
Saving
Finished 91 Baltimore.S3238QWX1
1 0.644630491733551
499 0.1327209323644638
Saving
1 0.6305994987487793
499 0.1273275911808014
Saving
1 0.6396521925926208
499 0.12524397671222687
Saving
1 0.6181713938713074
499 0.13084092736244202
1 0.6416339874267578
499 0.12529686093330383
Finished 92 Baltimore.S3278SOQ1
1 0.6303907632827759
499 0.14105573296546936
Saving
1 0.6378419399261475
499 0.14173255860805511
1 0.6455363631248474
499 0.13965126872062683
Saving
1 0.6436075568199158
499 0.13908514380455017
Saving
1 0.6353206634521484
499 0.13974156975746155
Finished 93 Baltimore.S3279JGV1
1 0.6394799947738647
499 0.11319178342819214
Saving
1 0.6434339284896851
499 0.10495912283658981
Saving
1 0.5987367033958435
499 0.1123950257897377
1 0.5948249101638794
499 0.11387306451797485
1 0.6379950046539307
499 0.1135818213224411
Finished 94 Baltimore.S3363WCM1
1 0.6357706785202026
499 0

499 0.12463612854480743
Finished 121 Baltimore.S4213KTC1
1 0.6183803081512451
499 0.12502412497997284
Saving
1 0.6218600273132324
499 0.12448486685752869
Saving
1 0.6247860789299011
499 0.12426912784576416
Saving
1 0.6207666397094727
499 0.12558843195438385
1 0.6063666343688965
499 0.12558984756469727
Finished 122 Baltimore.S4253TLX1
1 0.6387393474578857
499 0.1296595335006714
Saving
1 0.6308644413948059
499 0.12704969942569733
Saving
1 0.6214597821235657
499 0.129510298371315
1 0.6325816512107849
499 0.1284414529800415
1 0.6310373544692993
499 0.1296595335006714
Finished 123 Baltimore.S4310FQI1
1 0.5932967066764832
499 0.09967908263206482
Saving
1 0.6320399045944214
499 0.09931855648756027
Saving
1 0.601876437664032
499 0.09898897260427475
Saving
1 0.6013056635856628
499 0.10053045302629471
1 0.604229748249054
499 0.1005379781126976
Finished 124 Baltimore.S4393MHW1
1 0.6466804146766663
499 0.11949273198843002
Saving
1 0.6492785811424255
499 0.11875651031732559
Saving
1 0.6416267752647

...

499 0.11003505438566208
Saving
1 0.6100215911865234
499 0.11026985198259354
1 0.5918173789978027
499 0.11025063693523407
1 0.6056092977523804
499 0.11091701686382294
1 0.6063243746757507
499 0.11003892123699188
Finished 1123 Hartford.S6480AET3
1 0.6528288722038269
499 0.12486106157302856
Saving
1 0.6491444706916809
499 0.12235421687364578
Saving
1 0.6445707082748413
499 0.12539787590503693
1 0.648732602596283
499 0.12463359534740448
1 0.6520481109619141
499 0.12798519432544708
Finished 1124 Hartford.S6486OAH1
1 0.6117120981216431
499 0.10544226318597794
Saving
1 0.6116700172424316
499 0.10515791177749634
Saving
1 0.6187232732772827
499 0.10498234629631042
Saving
1 0.6109617948532104
499 0.10654519498348236
1 0.610680341720581
499 0.10491801053285599
Saving
Finished 1125 Hartford.S6508YJC2
1 0.613175630569458
499 0.11738687753677368
Saving
1 0.6138335466384888
499 0.11757417768239975
1 0.6164135932922363
499 0.11542176455259323
Saving
1 0.5939775109291077
499 0.11520704627037048
Saving


499 0.1248219832777977
Saving
1 0.6471952795982361
499 0.1265266239643097
1 0.6479458212852478
499 0.12645256519317627
1 0.6431980133056641
499 0.1267421841621399
1 0.6422743797302246
499 0.12285993993282318
Saving
Finished 1153 Hartford.S7709VXY1
1 0.6252647638320923
499 0.11814940720796585
Saving
1 0.6261856555938721
499 0.11808212101459503
Saving
1 0.6194417476654053
499 0.11807025223970413
Saving
1 0.6384307742118835
499 0.11814994364976883
1 0.6074428558349609
499 0.11798398941755295
Saving
Finished 1154 Hartford.S7795QIP1
1 0.6646373271942139
499 0.12325655668973923
Saving
1 0.6619547605514526
499 0.1280948966741562
1 0.660408079624176
499 0.12048601359128952
Saving
1 0.6615371704101562
499 0.12737256288528442
1 0.6618865132331848
499 0.12645690143108368
Finished 1155 Hartford.S7812MNP1
1 0.6822022199630737
499 0.14168494939804077
Saving
1 0.6792466044425964
499 0.1406605839729309
Saving
1 0.6831013560295105
499 0.14126648008823395
1 0.6836198568344116
499 0.1420055627822876
1 0.

499 0.10699883103370667
1 0.6184899806976318
499 0.10804803669452667
1 0.6150368452072144
499 0.10771030932664871
1 0.6333011388778687
499 0.10841862857341766
Finished 1183 Hartford.S8408LNV1
1 0.664337694644928
499 0.12548378109931946
Saving
1 0.6567174792289734
499 0.1254945546388626
1 0.6530138850212097
499 0.1251259744167328
Saving
1 0.6555331349372864
499 0.1256306916475296
1 0.6524255275726318
499 0.1254744678735733
Finished 1184 Hartford.S8427LIM1
1 0.6599753499031067
499 0.1283799558877945
Saving
1 0.6608276963233948
499 0.1277046501636505
Saving
1 0.6563274264335632
499 0.12782560288906097
1 0.6542713642120361
499 0.12719859182834625
Saving
1 0.6533989906311035
499 0.12823881208896637
Finished 1185 Hartford.S8435KQO1
1 0.6527067422866821
499 0.13433878123760223
Saving
1 0.6629818081855774
499 0.13475368916988373
1 0.656989574432373
499 0.1349828541278839
1 0.6629844903945923
499 0.140549436211586
1 0.6646608710289001
499 0.1344786435365677
Finished 1186 Hartford.S8440HOR1
1 0.

499 0.11289869248867035
Saving
1 0.5785624384880066
499 0.11282693594694138
Saving
Finished 1213 Hartford.S9358LVU1
1 0.6534733772277832
499 0.12197590619325638
Saving
1 0.6582714915275574
499 0.12984420359134674
1 0.6500291228294373
499 0.13201677799224854
1 0.6583763360977173
499 0.12844666838645935
1 0.6524964570999146
499 0.13187885284423828
Finished 1214 Hartford.S9403UJQ1
1 0.583615243434906
499 0.09716076403856277
Saving
1 0.5895612239837646
499 0.09712257981300354
Saving
1 0.6064857840538025
499 0.0971948429942131
1 0.5761947631835938
499 0.09711173921823502
Saving
1 0.5842956304550171
499 0.09712852537631989
Finished 1215 Hartford.S9438IKO1
1 0.5813139081001282
499 0.09825609624385834
Saving
1 0.5991639494895935
499 0.09815657883882523
Saving
1 0.5985092520713806
499 0.0981566458940506
1 0.613596498966217
499 0.09811978787183762
Saving
1 0.6357076168060303
499 0.09804891049861908
Saving
Finished 1216 Hartford.S9445CEO1
1 0.645446240901947
499 0.11518063396215439
Saving
1 0.645

499 0.12001657485961914
Saving
Finished 1243 Hartford.S9979MDO1
1 0.6330194473266602
499 0.14441078901290894
Saving
1 0.6443509459495544
499 0.14376170933246613
Saving
1 0.6486191749572754
499 0.14424410462379456
1 0.6596174836158752
499 0.15065951645374298
1 0.6515315175056458
499 0.14545071125030518
Finished 1244 Hartford.S9996WBC3
Complete


In [12]:
import pickle

pickle.dump(bases, open('/home/anton/Documents/Tulane/Research/BSNIP/AngleBasis1.pkl', 'wb'))

print('Complete')

Complete
